### Extract RL samples from csv file to pandas dataframe

- Read clean data from csv file.
- Generate coloumns of 
  'state', 'action', 'next_state', 'reward'
- Save results to csv or feather-format


In [ ]:
# !pip install feather-format
# !pip install sklearn
# !pip install --upgrade pandas

In [1]:
import os
import numpy as np
#from scipy.io import loadmat  # this is the SciPy module that loads mat-files
#from scipy import stats  # stats.zscore(a, axis=1, ddof=1) 
#import matplotlib.pyplot as plt
from datetime import datetime, date, time
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import pickle

In [2]:
os.getcwd()

'/home/ubuntu/mediRL/tuple_sars'

In [2]:
DATA_DIR = '../data'
INPUT_FILE = 'MIMICtable.csv'
OUTPUT_FILE = 'rl_samples.pkl'
infile = os.path.join(DATA_DIR, INPUT_FILE)
outfile = os.path.join(DATA_DIR, OUTPUT_FILE)
print(infile)
print(outfile)

../data/MIMICtable.csv
../data/rl_samples.pkl


In [3]:
cols = 'bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,delay_end_of_record_and_discharge_or_death,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,Temp_C,FiO2_1,Potassium,Sodium,Chloride,Glucose,BUN,Creatinine,Magnesium,Calcium,Ionised_Ca,CO2_mEqL,SGOT,SGPT,Total_bili,Albumin,Hb,WBC_count,Platelets_count,PTT,PT,INR,Arterial_pH,paO2,paCO2,Arterial_BE,Arterial_lactate,HCO3,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS'.split(',')
types = dict((n, np.float64) for n in cols)
#int_cols = 'bloc,icustayid,gender,age,elixhauser,re_admission,die_in_hosp,died_within_48h_of_out_time,mortality_90d'.split(',')
#for n in int_cols:
#    types[n] = np.int64
mimic3 = pd.read_csv(infile, header=0, usecols=cols, dtype=types)  # type(mimic3)
mimic3.head()
# check if there is any NaN
# pd.isnull(mimic3).any(1).to_numpy().nonzero()[0]

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS
0,1.0,3.0,7.245486e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,210.000000,0.0,0.0,6527.0,50.0,13617.0,520.0,-7090.0,4.0,1.0
1,2.0,3.0,7.245500e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,210.000000,0.0,0.0,6577.0,50.0,13982.0,365.0,-7405.0,5.0,2.0
2,3.0,3.0,7.245515e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,228.260870,0.0,0.0,6627.0,50.0,14262.0,280.0,-7635.0,5.0,2.0
3,4.0,3.0,7.245529e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,233.333333,0.0,0.0,6677.0,50.0,14602.0,340.0,-7925.0,5.0,1.0
4,5.0,3.0,7.245544e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,233.333333,0.0,0.0,6727.0,50.0,15142.0,540.0,-8415.0,5.0,2.0


In [4]:
# prepare data remove patients who died in ICU during data collection period # Nature paper use '&' and remove 850 only
id_ICU_died = mimic3.query('died_within_48h_of_out_time == 1 or delay_end_of_record_and_discharge_or_death < 24').icustayid.unique()

id_ICU_died.shape
# (3322,)

rawdata = mimic3[~mimic3.icustayid.isin(id_ICU_died)]

rawdata.shape

len(rawdata.icustayid.unique())
# 17621

# DID not updated the following yet

17621

In [5]:
# 47 columns of state variables
bin_cols = ['gender','mechvent','max_dose_vaso','re_admission']
norm_cols = ['age','Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1', 'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium','Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2', 'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index','PaO2_FiO2','cumulated_balance']
log_cols = ['SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR','input_total','input_4hourly','output_total','output_4hourly']

# action variables
# 'input_4hourly'
# 'max_dose_vaso'

In [6]:
icustayid_list = mimic3['icustayid'] # or just mimic3.icustayid; type(icustayid_list)
len(icustayid_list.unique())

20943

In [7]:
# state variables 47 
# raw values
states_raw = mimic3[ bin_cols + norm_cols + log_cols ]
states_raw.head()

# normalized for clustering in L2 distance
states_norm = pd.DataFrame(data=None, columns=states_raw.columns, index=states_raw.index)
# in python for loop is faster than pandas.DataFrame.apply()
for col in bin_cols:
    states_norm[col] = states_raw[col] - 0.5
for col in norm_cols:
    df = states_raw[col]
    states_norm[col] = (df - df.mean())/df.std()
for col in log_cols:
    df = np.log(0.1+states_raw[col])
    states_norm[col] = (df - df.mean())/df.std()

# max dose norad
states_norm['max_dose_vaso'] = np.log(states_norm['max_dose_vaso']+.6)
# increase the weight of variable 'input_4hourly'
# (bin_cols + norm_cols + log_cols)[44]
states_norm['input_4hourly'] = 2 * states_norm['input_4hourly']

states_norm.shape
# (278598, 47)

(278598, 47)

In [8]:
# hyper-parameters
n_models = 3       # 500 
test_frac = 0.2
km_frac = 0.25
#
n_states = 750   # number of states if discretized
n_act = [5,5]  # number of actions in each dimension if discretized
n_actions = np.prod(n_act) # number of actions if discretized

In [9]:
#########################################
##
##  discreteize continuous action space
##

# Two actions: IV fluid: 'input_4hourly' and Vasopressor: 'max_dose_vaso'
# mimic3['input_4hourly'].describe()
# mimic3['max_dose_vaso'].describe()

iv_bins = [-np.inf,0,50,180,530,np.inf]
vaso_bins = [-np.inf,0,0.08,0.22,0.45,np.inf]

mimic3['act_iv'] = pd.cut(mimic3['input_4hourly'], bins=iv_bins, labels=range(5))
mimic3['act_vaso'] =pd.cut(mimic3['max_dose_vaso'], bins=vaso_bins, labels=range(5))

# generate action code = (act_vi * 5 + act_vaso)
# mimic3['action'] = list(zip(mimic3.act_vi, mimic3.act_vaso))
mimic3['action'] = mimic3['act_iv'].cat.codes * 5 + mimic3['act_vaso'].cat.codes
mimic3[['act_iv','act_vaso','action']].head()

,act_iv,act_vaso,action
0,1,0,5
1,1,0,5
2,1,0,5
3,1,0,5
4,1,0,5


In [10]:
#########################################
##
##       preparing reward
##
##       use later reward design
## 

# Y90 # 'died_in_hosp' 8, 'mortality_90d' 10
# 100 for survival; -100 for non-survival
# mimic3['reward'] = (2 * (1 - mimic3['mortality_90d']) - 1)*100
# mimic3['reward'].head()

In [11]:
#for ii in range(n_models): 
# try n_models and choose the best one
#    train_ids, test_ids = train_test_split(mimic3['icustayid'].unique(), test_size = test_ratio)
#    train = mimic3[mimic3['icustayid'].isin(train_ids)]
#    test = mimic3[mimic3['icustayid'].isin(test_ids)]

In [12]:
train_ids, test_ids = train_test_split(mimic3.icustayid.unique(), test_size = test_frac)
states_norm_train = states_norm[mimic3.icustayid.isin(train_ids)]

In [13]:
## discretize continuous state space using clustering
## TODO: Timing
## states: 0 ~ 749 
## later add two absorbing states: -1 (die in hospital), 750 (discharge)
#states_kmeans = KMeans(n_clusters=n_states, max_iter=10000).fit(states_norm.sample(frac=km_frac))

## dump pickle file
#with open('states_kmeans.pkl', 'wb') as file:
#    pickle.dump(states_kmeans, file)

In [14]:
# load pickle file
infile_kmeans = os.path.join(DATA_DIR, 'states_kmeans.pkl')
states_kmeans = pickle.load(open(infile_kmeans, 'rb'))

/Users/elynn/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
mimic3['step'] = 1
mimic3['state'] = states_kmeans.predict(states_norm)
mimic3['next_state'] = 750  # default assign successful discharge 
mimic3['trajectory'] = 0 # default trajectory
mimic3['done'] = False
mimic3.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,SOFA,SIRS,act_iv,act_vaso,action,step,state,next_state,trajectory,done
0,1.0,3.0,7.245486e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,4.0,1.0,1,0,5,1,365,750,0,False
1,2.0,3.0,7.245500e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,462,750,0,False
2,3.0,3.0,7.245515e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,365,750,0,False
3,4.0,3.0,7.245529e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,1.0,1,0,5,1,365,750,0,False
4,5.0,3.0,7.245544e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,365,750,0,False


In [16]:
# may not be necessary if produced in order
mimic3.sort_values(by=['icustayid','bloc'], inplace=True)
mimic3.head(8)

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,SOFA,SIRS,act_iv,act_vaso,action,step,state,next_state,trajectory,done
0,1.0,3.0,7.245486e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,4.0,1.0,1,0,5,1,365,750,0,False
1,2.0,3.0,7.245500e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,462,750,0,False
2,3.0,3.0,7.245515e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,365,750,0,False
3,4.0,3.0,7.245529e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,1.0,1,0,5,1,365,750,0,False
4,5.0,3.0,7.245544e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,365,750,0,False
5,6.0,3.0,7.245558e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,1,0,5,1,365,750,0,False
6,9.0,3.0,7.245601e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,13.0,0.0,0,0,0,1,338,750,0,False
7,1.0,11.0,6.898241e+09,1.0,30766.069028,6.0,1.0,0.0,0.0,0.0,...,12.0,0.0,0,0,0,1,313,750,0,False


In [17]:
'''
Reward engineering
Info. Sources:
[1] https://www.mayoclinic.org/diseases-conditions/high-blood-pressure/in-depth/blood-pressure/art-20050982
[2] https://medlineplus.gov/ency/article/003855.htm
[3] Arterial base excess: https://en.wikipedia.org/wiki/Base_excess
[4] PaO2_FiO2: https://litfl.com/pao2-fio2-ratio/
[5] INR: https://www.mayoclinic.org/tests-procedures/prothrombin-time/about/pac-20384661
[6] Blood clotting test: PTT, PT, INR: https://emedicine.medscape.com/article/2086058-overview#a1
[7] Hemoglobin test: https://www.mayoclinic.org/tests-procedures/hemoglobin-test/about/pac-20385075
[8] Sepsis: https://www.nursingcenter.com/ncblog/march-2017/laboratory-signs-of-sepsis
'''

# ['dangerous_low', 'normal_low', 'normal_high', 'dangerous_high']
# currently I only use ['normal_low', 'normal_high'] as the normal range, out of which will be penalized
# with some relaxations 
# need help from critical care givers

VITAL_RANGES = {
'HR': [np.nan, 60, 100, 130],
'SysBP': [np.nan, 90, 140, 180], # Systolic blood pressure 
'MeanBP': [np.nan, 70, 100, np.nan], 
'DiaBP': [np.nan, 60, 90, 120], # Diastolic blood pressure
'RR': [np.nan, 12, 20, 30], # Respiratory Rate
'SpO2': [90, 95, 100, np.nan], # O2 saturation pulseoxymetry, Blood oxygen levels below 80 percent may compromise organ function, such as the brain and heart, and should be promptly addressed.
'Temp_C': [np.nan, 36.1, 38, np.nan], 
'FiO2_1': [], # fraction of inspired oxygen
'Potassium': [np.nan, 3.6, 5.2, 6],  # mmol/L
'Sodium': [np.nan, 135, 145, np.nan], # mEq/L
'Chloride': [np.nan, 97, 107, np.nan], # mEq/L
'Glucose': [np.nan, 100, 125, np.nan ], # mg/dL
'BUN': [np.nan, 7, 20, np.nan], # Blood urea nitrogen
'Creatinine': [np.nan, 0.55, 1.15, np.nan], 
'Magnesium': [np.nan, 1.5, 2.5, np.nan], 
'Calcium': [np.nan, 8.5, 10.5, np.nan], 
'Ionised_Ca': [],  #  In adults, a level of 4.64 to 5.28 milligrams per deciliter (mg/dL) is normal, but this range does not match the data
'CO2_mEqL': [np.nan, 23, 29, np.nan], 
'SGOT': [np.nan, 5, 40, np.nan], 
'SGPT': [np.nan, 7, 56, np.nan], 
'Total_bili': [np.nan, 1.71, 20.5, np.nan], # Total Bilirubin
'Albumin': [np.nan, 3, 6, np.nan], 
'Hb': [np.nan, 12, 1.5, np.nan], # Hemoglobin, For men, 13.5 to 17.5 grams per deciliter. For women, 12.0 to 15.5 grams per deciliter.
'WBC_count': [np.nan, 4.5, 11, np.nan], 
'Platelets_count': [np.nan, 150, 450, np.nan], 
'PTT': [np.nan, 40, 80, np.nan], # Partial thromboplastin time. The reference range of the PTT is 60-70 seconds.
'PT': [np.nan, 9, 15, np.nan], # prothrombin time
'INR': [np.nan, 0.5, 1.5, np.nan], # International normalized ratio. If you are not taking blood thinning medicines, such as warfarin, the normal range for your PT results is: 11 to 13.5 seconds. INR of 0.8 to 1.1. 
'Arterial_pH': [7.3, 7.35, 7.45, np.nan], 
'paO2': [np.nan, 75, 100, np.nan], # Partial pressure of oxygen
'paCO2': [np.nan, 38, 42, np.nan], # Partial pressure of carbon dioxide
'Arterial_BE': [np.nan, -2, 2, np.nan], # Arterial base excess
'Arterial_lactate': [np.nan, 0.5, 2.5, 4], 
'HCO3': [np.nan, 22, 28, np.nan], # mEq/L Bicarbonate
#'PaO2_FiO2': [100, 300, 2900, np.nan] # max value in the dataset is 2890.4761904761895. # PF ratio <333 mmHg if age <50y or PF ratio <250mmHg if age >50y; the Berlin definition of ARDS (PF ratio <300mmHg), and correlates with mortality: mortality 27% if (200, 300], 32% if (100, 200], 45% if (0, 100].  # OverflowError: math range error
# ---------------------------
#'SaO2':[np.nan, 0.94, 1, np.nan]
}

#IMPORTANT_VITALS = 'Arterial_lactate, Creatinine, Total_bili, Glucose, WBC_count, Platelets_count, PTT, PT, INR'.split(',') # PaO2_FiO2, 
IMPORTANT_VITALS = 'HR,SysBP,MeanBP,DiaBP,RR,SpO2,Arterial_lactate,Creatinine,Total_bili,Glucose,WBC_count,Platelets_count,PTT,PT,INR'.split(',') # PaO2_FiO2, 


# First time only use a small number of criterions. 
VITAL_IMPORTANCE = dict.fromkeys(VITAL_RANGES.keys(), 0)

for vitals in IMPORTANT_VITALS: 
    VITAL_IMPORTANCE[vitals] = 1

In [20]:
VITAL_IMPORTANCE

{'HR': 1,
 'SysBP': 1,
 'MeanBP': 1,
 'DiaBP': 1,
 'RR': 1,
 'SpO2': 1,
 'Temp_C': 0,
 'FiO2_1': 0,
 'Potassium': 0,
 'Sodium': 0,
 'Chloride': 0,
 'Glucose': 1,
 'BUN': 0,
 'Creatinine': 1,
 'Magnesium': 0,
 'Calcium': 0,
 'Ionised_Ca': 0,
 'CO2_mEqL': 0,
 'SGOT': 0,
 'SGPT': 0,
 'Total_bili': 1,
 'Albumin': 0,
 'Hb': 0,
 'WBC_count': 1,
 'Platelets_count': 1,
 'PTT': 1,
 'PT': 1,
 'INR': 1,
 'Arterial_pH': 0,
 'paO2': 0,
 'paCO2': 0,
 'Arterial_BE': 0,
 'Arterial_lactate': 1,
 'HCO3': 0}

In [ ]:
#mimic3.loc[[1,2], list(VITAL_RANGES.keys())].head(50)

In [ ]:
##
## iterate over dictionary with keys
##

# d = VITAL_RANGES
# for key in d:
#     print("{} = {}".format(key, d[key]))


In [ ]:
#vit = 'HR'
#VITAL_IMPORTANCE[vit]

In [ ]:
# generate 'trajectory' and 'next_state' and ...
# 0 - 749 cluster
# ... add two absorbing states: 751 (die in hospital), 750 (discharge)

DECEASED = 751
DECEASED_REWARD = -50
DISCHARGED = 750
SURVIVAL_REWARD = 50

#VITS = 'HR SysBP MeanBP DiaBP RR SpO2'.split()

def getVitalReward(row, regulateVitals, vital_weights):
    total_reward = 0
    #for vit in VITS:
    for vit in regulateVitals:
        if len(regulateVitals[vit]) > 2:
            lower = regulateVitals[vit][1]
            upper = regulateVitals[vit][2]
            mid = (upper - lower)/2.0
            importance = vital_weights[vit] # regulateVitals[vit][4]
            diff2 = float(upper-lower)/2
            scale = 1/((math.exp(diff2)-1)/(1+math.exp(diff2)))
            val = row[vit] 
            if (val > upper) or (val < (lower)):
                reward = -0.125*importance*scale * \
                    (1/(1+math.exp(lower-val)) - 1/(1+math.exp(upper-val)) - (1/scale))
                total_reward -= reward
    return total_reward

trajectory = 1  # one trajectory for one patient
step = 1  # the number of step in each trajectory

print('Processing patient trajectory #' + str(trajectory))

for i, row in mimic3.iterrows():

    mimic3.loc[i, 'trajectory'] = trajectory

    if i+1 == mimic3.shape[0]:
        mimic3.loc[i, 'step'] = step # no next df line
        mimic3.loc[i, 'next_state'] = DECEASED if row['died_in_hosp'] == 1 else DISCHARGED
        mimic3.loc[i, 'done'] = True
        mimic3.loc[i, 'reward'] = DECEASED_REWARD if row['mortality_90d'] == 1 else SURVIVAL_REWARD
        print('End of dataframe at row ' + str(i))
    elif mimic3.loc[(i+1), 'bloc'] > mimic3.loc[i, 'bloc']:
        mimic3.loc[i, 'step'] = step
        step += 1 # stay in the same trajectory, step ++ 
        mimic3.loc[i, 'next_state'] = mimic3.loc[i+1, 'state']
        mimic3.loc[i, 'reward'] = getVitalReward(mimic3.loc[i+1, :], VITAL_RANGES, VITAL_IMPORTANCE)
    elif mimic3.iloc[i+1]['bloc'] == 1:
        mimic3.loc[i, 'step'] = step
        step = 1 # prepare for next trajectory
        mimic3.loc[i, 'next_state'] = DECEASED if row['died_in_hosp'] == 1 else DISCHARGED
        mimic3.loc[i, 'done'] = True
        mimic3.loc[i, 'reward'] = DECEASED_REWARD if row['mortality_90d'] == 1 else SURVIVAL_REWARD
        trajectory = trajectory + 1
        print('Processing trajectory #' + str(trajectory))
    else:
        print('Error at entry' + str(i))

with open('rl_samples.pkl', 'wb') as file:
    pickle.dump(mimic3, file)

Processing patient trajectory #1
Processing trajectory #2
Processing trajectory #3
Processing trajectory #4
Processing trajectory #5
Processing trajectory #6
Processing trajectory #7
Processing trajectory #8
Processing trajectory #9
Processing trajectory #10
Processing trajectory #11
Processing trajectory #12
Processing trajectory #13
Processing trajectory #14
Processing trajectory #15
Processing trajectory #16
Processing trajectory #17
Processing trajectory #18
Processing trajectory #19
Processing trajectory #20
Processing trajectory #21
Processing trajectory #22
Processing trajectory #23
Processing trajectory #24
Processing trajectory #25
Processing trajectory #26
Processing trajectory #27
Processing trajectory #28
Processing trajectory #29
Processing trajectory #30
Processing trajectory #31
Processing trajectory #32
Processing trajectory #33
Processing trajectory #34
Processing trajectory #35
Processing trajectory #36
Processing trajectory #37
Processing trajectory #38
Processing tr

Processing trajectory #309
Processing trajectory #310
Processing trajectory #311
Processing trajectory #312
Processing trajectory #313
Processing trajectory #314
Processing trajectory #315
Processing trajectory #316
Processing trajectory #317
Processing trajectory #318
Processing trajectory #319
Processing trajectory #320
Processing trajectory #321
Processing trajectory #322
Processing trajectory #323
Processing trajectory #324
Processing trajectory #325
Processing trajectory #326
Processing trajectory #327
Processing trajectory #328
Processing trajectory #329
Processing trajectory #330
Processing trajectory #331
Processing trajectory #332
Processing trajectory #333
Processing trajectory #334
Processing trajectory #335
Processing trajectory #336
Processing trajectory #337
Processing trajectory #338
Processing trajectory #339
Processing trajectory #340
Processing trajectory #341
Processing trajectory #342
Processing trajectory #343
Processing trajectory #344
Processing trajectory #345
P

Processing trajectory #613
Processing trajectory #614
Processing trajectory #615
Processing trajectory #616
Processing trajectory #617
Processing trajectory #618
Processing trajectory #619
Processing trajectory #620
Processing trajectory #621
Processing trajectory #622
Processing trajectory #623
Processing trajectory #624
Processing trajectory #625
Processing trajectory #626
Processing trajectory #627
Processing trajectory #628
Processing trajectory #629
Processing trajectory #630
Processing trajectory #631
Processing trajectory #632
Processing trajectory #633
Processing trajectory #634
Processing trajectory #635
Processing trajectory #636
Processing trajectory #637
Processing trajectory #638
Processing trajectory #639
Processing trajectory #640
Processing trajectory #641
Processing trajectory #642
Processing trajectory #643
Processing trajectory #644
Processing trajectory #645
Processing trajectory #646
Processing trajectory #647
Processing trajectory #648
Processing trajectory #649
P

Processing trajectory #917
Processing trajectory #918
Processing trajectory #919
Processing trajectory #920
Processing trajectory #921
Processing trajectory #922
Processing trajectory #923
Processing trajectory #924
Processing trajectory #925
Processing trajectory #926
Processing trajectory #927
Processing trajectory #928
Processing trajectory #929
Processing trajectory #930
Processing trajectory #931
Processing trajectory #932
Processing trajectory #933
Processing trajectory #934
Processing trajectory #935
Processing trajectory #936
Processing trajectory #937
Processing trajectory #938
Processing trajectory #939
Processing trajectory #940
Processing trajectory #941
Processing trajectory #942
Processing trajectory #943
Processing trajectory #944
Processing trajectory #945
Processing trajectory #946
Processing trajectory #947
Processing trajectory #948
Processing trajectory #949
Processing trajectory #950
Processing trajectory #951
Processing trajectory #952
Processing trajectory #953
P

Processing trajectory #1213
Processing trajectory #1214
Processing trajectory #1215
Processing trajectory #1216
Processing trajectory #1217
Processing trajectory #1218
Processing trajectory #1219
Processing trajectory #1220
Processing trajectory #1221
Processing trajectory #1222
Processing trajectory #1223
Processing trajectory #1224
Processing trajectory #1225
Processing trajectory #1226
Processing trajectory #1227
Processing trajectory #1228
Processing trajectory #1229
Processing trajectory #1230
Processing trajectory #1231
Processing trajectory #1232
Processing trajectory #1233
Processing trajectory #1234
Processing trajectory #1235
Processing trajectory #1236
Processing trajectory #1237
Processing trajectory #1238
Processing trajectory #1239
Processing trajectory #1240
Processing trajectory #1241
Processing trajectory #1242
Processing trajectory #1243
Processing trajectory #1244
Processing trajectory #1245
Processing trajectory #1246
Processing trajectory #1247
Processing trajector

...

Processing trajectory #4144
Processing trajectory #4145
Processing trajectory #4146
Processing trajectory #4147
Processing trajectory #4148
Processing trajectory #4149
Processing trajectory #4150
Processing trajectory #4151
Processing trajectory #4152
Processing trajectory #4153
Processing trajectory #4154
Processing trajectory #4155
Processing trajectory #4156
Processing trajectory #4157
Processing trajectory #4158
Processing trajectory #4159
Processing trajectory #4160
Processing trajectory #4161
Processing trajectory #4162
Processing trajectory #4163
Processing trajectory #4164
Processing trajectory #4165
Processing trajectory #4166
Processing trajectory #4167
Processing trajectory #4168
Processing trajectory #4169
Processing trajectory #4170
Processing trajectory #4171
Processing trajectory #4172
Processing trajectory #4173
Processing trajectory #4174
Processing trajectory #4175
Processing trajectory #4176
Processing trajectory #4177
Processing trajectory #4178
Processing trajector

Processing trajectory #4437
Processing trajectory #4438
Processing trajectory #4439
Processing trajectory #4440
Processing trajectory #4441
Processing trajectory #4442
Processing trajectory #4443
Processing trajectory #4444
Processing trajectory #4445
Processing trajectory #4446
Processing trajectory #4447
Processing trajectory #4448
Processing trajectory #4449
Processing trajectory #4450
Processing trajectory #4451
Processing trajectory #4452
Processing trajectory #4453
Processing trajectory #4454
Processing trajectory #4455
Processing trajectory #4456
Processing trajectory #4457
Processing trajectory #4458
Processing trajectory #4459
Processing trajectory #4460
Processing trajectory #4461
Processing trajectory #4462
Processing trajectory #4463
Processing trajectory #4464
Processing trajectory #4465
Processing trajectory #4466
Processing trajectory #4467
Processing trajectory #4468
Processing trajectory #4469
Processing trajectory #4470
Processing trajectory #4471
Processing trajector

Processing trajectory #4730
Processing trajectory #4731
Processing trajectory #4732
Processing trajectory #4733
Processing trajectory #4734
Processing trajectory #4735
Processing trajectory #4736
Processing trajectory #4737
Processing trajectory #4738
Processing trajectory #4739
Processing trajectory #4740
Processing trajectory #4741
Processing trajectory #4742
Processing trajectory #4743
Processing trajectory #4744
Processing trajectory #4745
Processing trajectory #4746
Processing trajectory #4747
Processing trajectory #4748
Processing trajectory #4749
Processing trajectory #4750
Processing trajectory #4751
Processing trajectory #4752
Processing trajectory #4753
Processing trajectory #4754
Processing trajectory #4755
Processing trajectory #4756
Processing trajectory #4757
Processing trajectory #4758
Processing trajectory #4759
Processing trajectory #4760
Processing trajectory #4761
Processing trajectory #4762
Processing trajectory #4763
Processing trajectory #4764
Processing trajector

Processing trajectory #5023
Processing trajectory #5024
Processing trajectory #5025
Processing trajectory #5026
Processing trajectory #5027
Processing trajectory #5028
Processing trajectory #5029
Processing trajectory #5030
Processing trajectory #5031
Processing trajectory #5032
Processing trajectory #5033
Processing trajectory #5034
Processing trajectory #5035
Processing trajectory #5036
Processing trajectory #5037
Processing trajectory #5038
Processing trajectory #5039
Processing trajectory #5040
Processing trajectory #5041
Processing trajectory #5042
Processing trajectory #5043
Processing trajectory #5044
Processing trajectory #5045
Processing trajectory #5046
Processing trajectory #5047
Processing trajectory #5048
Processing trajectory #5049
Processing trajectory #5050
Processing trajectory #5051
Processing trajectory #5052
Processing trajectory #5053
Processing trajectory #5054
Processing trajectory #5055
Processing trajectory #5056
Processing trajectory #5057
Processing trajector

Processing trajectory #5316
Processing trajectory #5317
Processing trajectory #5318
Processing trajectory #5319
Processing trajectory #5320
Processing trajectory #5321
Processing trajectory #5322
Processing trajectory #5323
Processing trajectory #5324
Processing trajectory #5325
Processing trajectory #5326
Processing trajectory #5327
Processing trajectory #5328
Processing trajectory #5329
Processing trajectory #5330
Processing trajectory #5331
Processing trajectory #5332
Processing trajectory #5333
Processing trajectory #5334
Processing trajectory #5335
Processing trajectory #5336
Processing trajectory #5337
Processing trajectory #5338
Processing trajectory #5339
Processing trajectory #5340
Processing trajectory #5341
Processing trajectory #5342
Processing trajectory #5343
Processing trajectory #5344
Processing trajectory #5345
Processing trajectory #5346
Processing trajectory #5347
Processing trajectory #5348
Processing trajectory #5349
Processing trajectory #5350
Processing trajector

In [ ]:
mimic3.head(20)

In [ ]:
i = 1
#mimic3.loc[i, 'reward'] = getVitReward(
#            mimic3.loc[i+1, :], VITAL_RANGES)

total_reward = 0

row = mimic3.loc[i+1, :]
regulateVitals = VITAL_RANGES
    #for vit in VITS:
for vit in regulateVitals:
    if len(regulateVitals[vit]) > 2:
        lower = regulateVitals[vit][1]
        upper = regulateVitals[vit][2]
        mid = (upper - lower)/2.0
        importance = 1 # regulateVitals[vit][4]
        diff2 = float(upper-lower)/2
        scale = 1/((math.exp(diff2)-1)/(1+math.exp(diff2)))
        val = row[vit] 
        if (val > upper) or (val < (lower)):
            reward = -0.125*importance*scale * \
                (1/(1+math.exp(lower-val)) - 1/(1+math.exp(upper-val)) - (1/scale))
            total_reward -= reward

In [ ]:
diff2

In [ ]:
scale = 1/((math.exp(diff2)-1)/(1+math.exp(diff2)))